In [3]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
py.sign_in('erikrood','3eqsrype8v')

In [192]:
PE_data = pandas.read_csv('/Users/erikrood/desktop/Sector_Margins_10_15_16.csv') 

In [193]:
PE_data.drop([0,1,2])


,Symbol,Company Name,Security Type,Share price,P_E,EPS,P_E_calc,Security Price,EBITD Margin (TTM),Market Capitalization,Market_cap_clean,Sector/Industry,Industry,"Revenue (TTM, Thousands)",count,Revenue_clean,Profit to mkt cap
3,EQIX,Equinix Inc,CSRT,360.25,403.10,0.89,404.78,351.2500,0.413642,$25.1B,"$25,100,000",Real Estate,Equity Real Estate Investment Trusts (REITs),3161.7770,1.0,"$3,161,777.00",0.000052
4,NFLX,Netflix Inc,CS,101.47,315.17,0.32,317.09,104.8650,0.584791,$45B,"$45,000,000",Consumer Discretionary,Internet & Direct Marketing Retail,7624.6280,1.0,"$7,624,628.00",0.000099
5,ALXN,Alexion Pharmaceuticals Inc,CS,118.34,299.70,0.39,303.44,125.1300,0.362509,$27.6B,"$27,600,000",Health Care,Biotechnology,2821.6580,1.0,"$2,821,658.00",0.000037
6,INCY,Incyte Corp,CS,87.10,256.82,0.34,256.18,97.9700,0.207029,$17.8B,"$17,800,000",Health Care,Biotechnology,941.2430,1.0,"$941,243.00",0.000011
7,CRM,salesforce.com Inc,CS,74.27,239.30,0.31,239.58,70.9800,0.095010,$48.8B,"$48,800,000",Information Technology,Software,7474.5860,1.0,"$7,474,586.00",0.000015
8,NEM,Newmont Mining Corp,CS,34.23,218.48,0.16,213.94,34.0300,0.368733,$18.1B,"$18,100,000",Materials,Metals & Mining,7919.0000,1.0,"$7,919,000.00",0.000161
9,AMZN,Amazon.com Inc,CS,822.96,205.21,4.01,205.23,839.6500,0.092310,$398.9B,"$398,900,000",Consumer Discretionary,Internet & Direct Marketing Retail,120637.0000,1.0,"$120,637,000.00",0.000028
10,IMO,Imperial Oil Ltd,CS,32.41,121.46,0.27,120.04,32.1700,0.083035,$27.6B,"$27,600,000",Energy,"Oil, Gas & Consumable Fuels",17468.7855,1.0,"$17,468,785.50",0.000053
11,ZBH,Zimmer Biomet Holdings Inc,CS,127.15,114.33,1.11,114.55,131.1600,0.393838,$26.1B,"$26,100,000",Health Care,Health Care Equipment & Supplies,7533.8000,1.0,"$7,533,800.00",0.000114
12,SE,Spectra Energy Corp,CS,42.55,99.84,0.43,98.95,41.5600,0.500806,$29.1B,"$29,100,000",Energy,"Oil, Gas & Consumable Fuels",4962.0000,1.0,"$4,962,000.00",0.000085


In [194]:
PE_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 17 columns):
Symbol                      407 non-null object
Company Name                406 non-null object
Security Type               406 non-null object
Share price                 407 non-null float64
P_E                         358 non-null float64
EPS                         407 non-null float64
P_E_calc                    358 non-null float64
Security Price              406 non-null float64
EBITD Margin (TTM)          336 non-null float64
Market Capitalization       406 non-null object
Market_cap_clean            406 non-null object
Sector/Industry             406 non-null object
Industry                    406 non-null object
Revenue (TTM, Thousands)    406 non-null float64
count                       406 non-null float64
Revenue_clean               406 non-null object
Profit to mkt cap           406 non-null float64
dtypes: float64(9), object(8)
memory usage: 54.1+ KB


## General cleaning

In [195]:
#convert from non-null object to string
PE_data['P_E'] = PE_data['P_E'].astype(str)


In [196]:
#strip/remove/drop where the row is #N/A 
#(this means the company had negative earnings, creating a negative P/E -- it's therefore excluded from calculation)
PE_data = PE_data[PE_data.P_E != '#N/A']
PE_data = PE_data[PE_data.P_E != 'nan']
#converting the clean P_E values over to float (number) now that the N/A is stripped
PE_data['P_E'] = PE_data['P_E'].astype(float)

In [197]:
PE_data.head()

,Symbol,Company Name,Security Type,Share price,P_E,EPS,P_E_calc,Security Price,EBITD Margin (TTM),Market Capitalization,Market_cap_clean,Sector/Industry,Industry,"Revenue (TTM, Thousands)",count,Revenue_clean,Profit to mkt cap
0,GSK,Glaxosmithkline PLC,DR,41.68,3228.51,0.01,4168.00,42.970,NaN,$104.3B,"$104,300,000",Health Care,Pharmaceuticals,34570.144,1.0,"$34,570,144.00",0.000000
1,RIO,Rio Tinto PLC,DR,31.77,1246.86,0.03,1059.00,33.150,NaN,$59.7B,"$59,700,000",Materials,Metals & Mining,32349.000,1.0,"$32,349,000.00",0.000000
2,KMI,Kinder Morgan Inc.,CS,20.36,552.66,0.04,509.00,21.670,0.492764,$49.2B,"$49,200,000",Energy,"Oil, Gas & Consumable Fuels",13682.000,1.0,"$13,682,000.00",0.000137
3,EQIX,Equinix Inc,CSRT,360.25,403.10,0.89,404.78,351.250,0.413642,$25.1B,"$25,100,000",Real Estate,Equity Real Estate Investment Trusts (REITs),3161.777,1.0,"$3,161,777.00",0.000052
4,NFLX,Netflix Inc,CS,101.47,315.17,0.32,317.09,104.865,0.584791,$45B,"$45,000,000",Consumer Discretionary,Internet & Direct Marketing Retail,7624.628,1.0,"$7,624,628.00",0.000099


In [198]:
#used to replace spaces w/ "_" (strip whitespace)
PE_data.columns = [x.strip().replace(' ', '_') for x in PE_data.columns]
PE_data.columns = [x.strip().replace('[', '_') for x in PE_data.columns]
PE_data.columns = [x.strip().replace(']', '_') for x in PE_data.columns]
PE_data.columns = [x.strip().replace('/', '_') for x in PE_data.columns]
PE_data.columns = [x.strip().replace('(', '_') for x in PE_data.columns]
PE_data.columns = [x.strip().replace(')', '_') for x in PE_data.columns]

In [199]:
PE_data.head()

,Symbol,Company_Name,Security_Type,Share_price,P_E,EPS,P_E_calc,Security_Price,EBITD_Margin__TTM_,Market_Capitalization,Market_cap_clean,Sector_Industry,Industry,"Revenue__TTM,_Thousands_",count,Revenue_clean,Profit_to_mkt_cap
0,GSK,Glaxosmithkline PLC,DR,41.68,3228.51,0.01,4168.00,42.970,NaN,$104.3B,"$104,300,000",Health Care,Pharmaceuticals,34570.144,1.0,"$34,570,144.00",0.000000
1,RIO,Rio Tinto PLC,DR,31.77,1246.86,0.03,1059.00,33.150,NaN,$59.7B,"$59,700,000",Materials,Metals & Mining,32349.000,1.0,"$32,349,000.00",0.000000
2,KMI,Kinder Morgan Inc.,CS,20.36,552.66,0.04,509.00,21.670,0.492764,$49.2B,"$49,200,000",Energy,"Oil, Gas & Consumable Fuels",13682.000,1.0,"$13,682,000.00",0.000137
3,EQIX,Equinix Inc,CSRT,360.25,403.10,0.89,404.78,351.250,0.413642,$25.1B,"$25,100,000",Real Estate,Equity Real Estate Investment Trusts (REITs),3161.777,1.0,"$3,161,777.00",0.000052
4,NFLX,Netflix Inc,CS,101.47,315.17,0.32,317.09,104.865,0.584791,$45B,"$45,000,000",Consumer Discretionary,Internet & Direct Marketing Retail,7624.628,1.0,"$7,624,628.00",0.000099


## Setting up PE by sector dataframe

In [200]:
df = PE_data[[11,4]]
df[['P_E']] = df[['P_E']].astype(float)
df.head()

/Users/erikrood/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Sector_Industry,P_E
0,Health Care,3228.51
1,Materials,1246.86
2,Energy,552.66
3,Real Estate,403.10
4,Consumer Discretionary,315.17


In [201]:
#chart_1
average_PE_sector = df.groupby('Sector_Industry').mean().sort_values(by='P_E',ascending=False).round(decimals=0).reset_index()
average_PE_sector.head()

,Sector_Industry,P_E
0,Materials,107.0
1,Health Care,105.0
2,Energy,78.0
3,Real Estate,67.0
4,Consumer Discretionary,36.0


In [202]:
#chart_2
std_PE_sector = df.groupby('Sector_Industry').agg(np.std, ddof=0).sort_values(by='P_E',ascending=False).round(decimals=0).reset_index()
std_PE_sector.columns = ['Sector_Industry', 'PE_Std_dev'] #renaming columns
std_PE_sector.head()

,Sector_Industry,PE_Std_dev
0,Health Care,459.0
1,Materials,289.0
2,Energy,136.0
3,Real Estate,95.0
4,Consumer Discretionary,54.0


In [203]:
#Chart_2.5
j1 = average_PE_sector.merge(std_PE_sector, how='left', on="Sector_Industry")
j1.head(10)

,Sector_Industry,P_E,PE_Std_dev
0,Materials,107.0,289.0
1,Health Care,105.0,459.0
2,Energy,78.0,136.0
3,Real Estate,67.0,95.0
4,Consumer Discretionary,36.0,54.0
5,Information Technology,30.0,33.0
6,Consumer Staples,28.0,14.0
7,Industrials,22.0,11.0
8,Utilities,20.0,8.0
9,Telecommunication Services,20.0,8.0


In [204]:
#chart_3
median_PE_sector = df.groupby('Sector_Industry').median().sort_values(by='P_E',ascending=False).round(decimals=0).reset_index()
median_PE_sector.head()

,Sector_Industry,P_E
0,Real Estate,37.0
1,Energy,28.0
2,Health Care,27.0
3,Materials,25.0
4,Information Technology,24.0


## PE by company

In [205]:
#chart_4
df1 = PE_data[[1,4]]
#df1.head(100)
top_PE_company = df1.groupby('Company_Name').mean().sort_values(by='P_E',ascending=False).round(decimals=0).reset_index()
top_PE_company.head()
#top_10_PE_company = top_PE_company.head(10)

,Company_Name,P_E
0,Glaxosmithkline PLC,3229.0
1,Rio Tinto PLC,1247.0
2,Kinder Morgan Inc.,553.0
3,Equinix Inc,403.0
4,Netflix Inc,315.0


In [206]:
top_10_PE_company = top_PE_company.head(10)
top_10_PE_company['P_E'] = top_10_PE_company['P_E'].astype(int)
bottom_10_PE_company = top_PE_company.tail(10)
bottom_10_PE_company['P_E'] = bottom_10_PE_company['P_E'].astype(int)

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## EBITD margin by sector

In [207]:
#stripping those that don't contain EBITD values
PE_data['EBITD_Margin__TTM_'] = PE_data['EBITD_Margin__TTM_'].astype(str)
EBITD_data = PE_data[PE_data.EBITD_Margin__TTM_ != 'nan']
PE_data['EBITD_Margin__TTM_'] = PE_data['EBITD_Margin__TTM_'].astype(float)
EBITD_data['EBITD_Margin__TTM_'] = EBITD_data['EBITD_Margin__TTM_'].astype(float)
EBITD_data['EBITD_Margin__TTM_'] = EBITD_data['EBITD_Margin__TTM_']*100
EBITD_data.head()

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/erikrood/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Symbol,Company_Name,Security_Type,Share_price,P_E,EPS,P_E_calc,Security_Price,EBITD_Margin__TTM_,Market_Capitalization,Market_cap_clean,Sector_Industry,Industry,"Revenue__TTM,_Thousands_",count,Revenue_clean,Profit_to_mkt_cap
2,KMI,Kinder Morgan Inc.,CS,20.36,552.66,0.04,509.00,21.670,49.27642,$49.2B,"$49,200,000",Energy,"Oil, Gas & Consumable Fuels",13682.000,1.0,"$13,682,000.00",0.000137
3,EQIX,Equinix Inc,CSRT,360.25,403.10,0.89,404.78,351.250,41.36424,$25.1B,"$25,100,000",Real Estate,Equity Real Estate Investment Trusts (REITs),3161.777,1.0,"$3,161,777.00",0.000052
4,NFLX,Netflix Inc,CS,101.47,315.17,0.32,317.09,104.865,58.47908,$45B,"$45,000,000",Consumer Discretionary,Internet & Direct Marketing Retail,7624.628,1.0,"$7,624,628.00",0.000099
5,ALXN,Alexion Pharmaceuticals Inc,CS,118.34,299.70,0.39,303.44,125.130,36.25092,$27.6B,"$27,600,000",Health Care,Biotechnology,2821.658,1.0,"$2,821,658.00",0.000037
6,INCY,Incyte Corp,CS,87.10,256.82,0.34,256.18,97.970,20.70294,$17.8B,"$17,800,000",Health Care,Biotechnology,941.243,1.0,"$941,243.00",0.000011


In [208]:
df2 = EBITD_data[[11,8]]

In [209]:
#chart_5
average_EBITD_sector = df2.groupby('Sector_Industry').mean().sort_values(by='EBITD_Margin__TTM_',ascending=False).reset_index()
average_EBITD_sector.head()

,Sector_Industry,EBITD_Margin__TTM_
0,Real Estate,44.749670
1,Financials,40.461852
2,Utilities,35.396935
3,Telecommunication Services,33.203306
4,Information Technology,29.767919


## EBITD margin by company

In [210]:
#to include revenue:
##df3 = EBITD_data[[1,8,15]]
##average_EBITD_company = df3.groupby(['Company_Name','Revenue_clean']).mean().sort_values(by='EBITD_Margin__TTM_',ascending=False).reset_index()
df3 = EBITD_data[[1,8]]
average_EBITD_company = df3.groupby(['Company_Name']).mean().sort_values(by='EBITD_Margin__TTM_',ascending=False).reset_index()
average_EBITD_company.head(10)

,Company_Name,EBITD_Margin__TTM_
0,Mitsubishi UFJ Financial Group,87.12821
1,Visa Inc,69.91384
2,CME Group Inc,68.81269
3,Gilead Sciences Inc,67.69022
4,Taiwan Semiconductor Manufacturing Co Ltd,64.57556
5,American Tower Corp,60.13103
6,Netflix Inc,58.47908
7,Mastercard Inc,56.59173
8,U.S. Bancorp,55.64148
9,M&T Bank Corp,55.29635


In [211]:
#setting up top/bottom 10 for charts
top_10_EBITD_company = average_EBITD_company.head(10)
bottom_10_EBITD_company = average_EBITD_company.tail(10)

In [212]:
#bar chart
x = average_PE_sector['Sector_Industry']
y = average_PE_sector['P_E']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(23, 191, 99)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='average P/E ratio by sector',
    xaxis=dict(
        title='sector',
        tickangle = 47,
    ),
    yaxis=dict(
        title='avg. P/E ratio'
    ),
     margin=go.Margin(
        b = 220
        
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='average_PE_sector')

In [213]:

#bar chart
x = median_PE_sector['Sector_Industry']
y = median_PE_sector['P_E']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(255, 170, 15)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='median P/E ratio by sector',
    xaxis=dict(
        title='sector',
        tickangle = 47,
    ),
    yaxis=dict(
        title='median P/E ratio'
    ),
     margin=go.Margin(
        b = 220
        
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='median_PE_sector')

In [214]:
#combo chart - bar with avg PE, line with std dev

x = j1['Sector_Industry']
y1 = j1['P_E']
y2 = j1['PE_Std_dev']


trace0 = go.Bar(
        x=x,
        y=y1,
        name='Avg. P/E ratio',
        marker=dict(
            color='rgb(255, 217, 102)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.6
    )

trace1 = go.Scatter(
        x=x,
        y=y2,
        name='Standard deviation of P/E ratio',
        yaxis = 'y2',
        marker=dict(
            color='rgb(84, 226, 129)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
        
data = [trace0, trace1]
    
layout = go.Layout(
      title='mean P/E by sector, with standard deviation',
    legend=dict(
        x=.5,
        y=1,
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2,
        ),
    xaxis=dict(
        title='category',
        tickangle = 47,
    ),
    
    yaxis=dict(
        title='P/E ratio'
    ),
    
    yaxis2=dict(
        title='Standard deviation',
        overlaying='y',
        side='right'
    ),
    
     margin=go.Margin(
        b = 150,
        r=50,
        t = 50
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='mean_PE_plus_std')

In [215]:

#bar chart
x = top_10_PE_company['Company_Name']
y = top_10_PE_company['P_E']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(91, 184, 255)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='P/E ratio by company (top 10)',
    xaxis=dict(
        title='company',
        tickangle = 47,
    ),
    yaxis=dict(
        title='P/E ratio'
    ),
     margin=go.Margin(
        b = 220
        
    ),
    
    annotations=[
        dict(x=xi,y=yi,
             text=str(yi),
             xanchor='center',
             yanchor='bottom',
             showarrow=False,
        ) for xi, yi in zip(x, y)]

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PE_company_top_10')

In [216]:

#bar chart
x = bottom_10_PE_company['Company_Name']
y = bottom_10_PE_company['P_E']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(249, 98, 87)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='P/E ratio by company (bottom 10)',
    xaxis=dict(
        title='company',
        tickangle = 47,
    ),
    yaxis=dict(
        title='P/E ratio'
    ),
     margin=go.Margin(
        b = 220
        
    )
    
    ##annotations=[
    #    dict(x=xi,y=yi,
    #         text=str(yi),
    #         xanchor='center',
    #         yanchor='bottom',
    #         showarrow=False,
    #    ) for xi, yi in zip(x, y)]

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PE_company_bottom_10')

In [217]:
#bar chart
x = average_EBITD_sector['Sector_Industry']
y = average_EBITD_sector['EBITD_Margin__TTM_']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(23, 191, 99)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='average EBITD margin by sector',
    xaxis=dict(
        title='sector',
        tickangle = 47,
    ),
    yaxis=dict(
        title='avg. EBITD margin'
    ),
     margin=go.Margin(
        b = 220
        
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='average_ebitd_sector')

In [218]:
#bar chart
x = top_10_EBITD_company['Company_Name']
y = top_10_EBITD_company['EBITD_Margin__TTM_']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(91, 184, 255)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='EBITD margin by company (top 10)',
    xaxis=dict(
        title='company',
        tickangle = 47,
    ),
    yaxis=dict(
        title='EBITD margin'
    ),
     margin=go.Margin(
        b = 220
        
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='EBITD_Margin__TTM__company_top_10')

In [219]:
#bar chart
x = bottom_10_EBITD_company['Company_Name']
y = bottom_10_EBITD_company['EBITD_Margin__TTM_']

data = [
    go.Bar(
        x=x,
        y=y,
        
        marker=dict(
            color='rgb(249, 98, 87)',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5
            ),
        ),
        opacity=0.8
    )
]
layout = go.Layout(
      title='EBITD margin by company (bottom 10)',
    xaxis=dict(
        title='company',
        tickangle = 47,
    ),
    yaxis=dict(
        title='EBITD margin'
    ),
     margin=go.Margin(
        b = 220
        
    )

    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='EBITD_Margin__TTM__company_bottom_10')